# Standard Library Modules

In [1]:
import random

import time

# Third Party Modules

In [2]:
import gurobipy as gp
import networkx as nx
from networkx.algorithms.approximation.steinertree import steiner_tree as steiner_tree_2
import scipy
from scipy.optimize import newton
from scipy.optimize import root_scalar
import sympy
from statistics import fmean as mean

# Constants

## Global Constants

In [3]:
import GlobalConstants

## Debug Level Flags

In [4]:
import DebugConstants as db

# Helper Functions

# Generating Problem Instances

In [5]:
from MulticastPackingInstance import MulticastPackingInstance

# Create (Reduced) LP

# Column Generating Subproblem

## Simple 2-Approximate Steiner Tree Column Generator

In [6]:
from Approx2MulticastPackingColumnGenerator import Approx2MulticastPackingColumnGenerator

## Steiner Tree IP Solving Column Generator

In [7]:
from ExactMulticastPackingColumnGeneratorIP import ExactMulticastPackingColumnGeneratorIP

# Solvers

## Pure Column Generation Solver Class

In [8]:
from PureColGenMcpSolver import PureColGenMcpSolver

## Jansen Zhang 2008 Convex MinMax Solver

In [9]:
from JansenZhangMinMaxer import JansenZhangMinMaxer

# Testing

In [10]:
def test_instance_generation(num_tests):
    assert not is_connected(nx.empty_graph(NUM_NODES))
    assert is_connected(get_random_connected_graph())
    G = get_random_connected_graph()
    for i in range(num_tests):
        request = MulticastRequest(MAX_MULTICAST_SIZE, G)
        source = request.source
        recipients = request.recipients
        assert request.source is not None
        assert request.recipients
        assert not request.source in request.recipients
        assert isinstance(request.recipients, set)
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    for request in instance.requests:
        assert request.source in instance.graph
        assert request.recipients.issubset(instance.graph)

def test_LP_creation(num_tests):
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    reduced_LP = create_LP(instance.graph, instance.requests)
    assert len(reduced_LP.getVars()) == 1
    assert len(reduced_LP.getConstrs()) == NUM_EDGES + NUM_MULTICAST_REQUESTS, "Num of constraints: {} Expected: {}".format(len(reduced_LP.getConstrs()), NUM_EDGES + NUM_MULTICAST_REQUESTS)
    
def test_subproblem(num_tests):
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    reduced_LP = create_LP(instance.graph, instance.requests)
    column_generator = Approx2MulticastPackingColumnGenerator(instance, reduced_LP)
    
    new_trees = column_generator.generate_new_trees()
    assert (len(new_trees) == instance.num_requests)
    for tree in new_trees:
        assert cost(tree, 1) == len(tree.edges()), "cost: {} edges: {}".format(cost(tree, 1), len(tree.edges()))
           
    assert len(reduced_LP.getVars()) == 1+instance.num_requests
    
def run_tests(num_tests):
    test_instance_generation(num_tests)
    test_LP_creation(num_tests)
    test_subproblem(num_tests)

NUM_TESTS = 0
#run_tests(NUM_TESTS)

# Rounding Algorithms

## (Totally) Randomized Rounding

In [11]:
def randRoundingHelper(fracSoln):
    randNum = random.random()
    for T in fracSoln:
        randNum -= fracSoln[T]
        if randNum < 0:
            return T
    
    
def randRounding(MCPsolver):
    intSoln = list()
    fracSoln = MCPsolver.solution[MCPsolver.iteration-1]
    for i in range(MCPsolver.instance.num_requests):
        intSoln.append(FrozenDict({randRoundingHelper(fracSoln[i]) : 1}))
        
    return tuple(intSoln)

# Run the solver

In [12]:
for i in range(1):
    print("BEGIN NEW TEST")
    instance = MulticastPackingInstance(GlobalConstants.NUM_MULTICAST_REQUESTS, GlobalConstants.MAX_MULTICAST_SIZE)
    print("Column Generation - 2-approx")
    CGsolver = PureColGenMcpSolver(instance=instance)
    CGtimer = list()
    while(not CGsolver.stop_flag):
        prev_time = time.perf_counter()
        CGsolver.perform_iteration()
        new_time = time.perf_counter()
        CGtimer.append(new_time - prev_time)
    print()
    print("Average Time of Pure Column Generation Iteration with 2-approx block solver: {}".format(mean(CGtimer)))
    print()
    
    print("Column Generation - Exact")
    CGsolver2 = PureColGenMcpSolver(instance=instance, block_solver="Exact")
    CGtimer2 = list()
    while(not CGsolver2.stop_flag):
        prev_time = time.perf_counter()
        CGsolver2.perform_iteration()
        new_time = time.perf_counter()
        CGtimer2.append(new_time - prev_time)
    print()
    print("Average Time of Pure Column Generation Iteration with an exact block solver: {}".format(mean(CGtimer2)))
    print()
    
    print("Jansen-Zhang Algorithm")
    JZsolver = JansenZhangMinMaxer(instance=instance)
    JZtimer = list()
    while(not JZsolver.stop_flag):
        prev_time = time.perf_counter()
        JZsolver.perform_iteration()
        new_time = time.perf_counter()
        JZtimer.append(new_time - prev_time)
    print()
    print("Average Time of Jansen-Zhang Algorithm Iteration: {}".format(mean(JZtimer)))

BEGIN NEW TEST
Column Generation - 2-approx
Set parameter Username
0
lambda(x): 1.0
phi_t(x): 0.0
tolerance: 1.0
2
lambda(x): 0.6666666666666666
phi_t(x): -0.40546510810816444
tolerance: 0.0

Average Time of Pure Column Generation Iteration with 2-approx block solver: 0.00542363333333314

Column Generation - Exact
0
lambda(x): 1.0
phi_t(x): 0.0
tolerance: 1.0
5
lambda(x): 0.6666666666666666
phi_t(x): -0.40546510810816444
tolerance: 0.0

Average Time of Pure Column Generation Iteration with an exact block solver: 0.005322099999999737

Jansen-Zhang Algorithm
0
lambda(x): 1.0
phi_t(x): 0.141287145561784
tolerance: 0.92
100
lambda(x): 0.775740871856443
phi_t(x): -0.135234340006967
tolerance: 0.836491563178971
200
lambda(x): 0.667591503920206
phi_t(x): -0.248055826298516
tolerance: 0.134061746767725
300
lambda(x): 0.666763194321129
phi_t(x): -0.320791740195829
tolerance: 0.0581407374016145
400
lambda(x): 0.666780524761558
phi_t(x): -0.321185758671627
tolerance: 0.0496137152873965
500
lambda

In [13]:
x = fracSolnCG
z = roundedCG
for i in range(len(x)):
    for T in z[i]:
        print(T)
        print(z[i][T])
    for T in x[i]:
        print(T)
        print(x[i][T])


NameError: name 'fracSolnCG' is not defined

In [ ]:
fracSolnCG = CGsolver.solution[CGsolver.iteration-1]
fracSolnCG2 = CGsolver2.solution[CGsolver2.iteration-1]
#fracSolnJZ = MCPsolver.solution[MCPsolver.iteration-1]
roundedCG = randRounding(CGsolver)
roundedCG2 = randRounding(CGsolver2)
#roundedJZ = randRounding(JZsolver)
print(JZsolver.lamb(roundedCG))
print(JZsolver.lamb(roundedCG2))
#print(JZsolver.lamb(roundedJZ))